# Testing Underwater GAN (UGAN) in Colab

## Test tensorflow

In [0]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


## Prepare data

* download repo
* download all the data needed
* make any needed preparation of data

In [5]:
# clone repo
%cd /content/
!if ! test -d Underwater-Color-Correction/; then git clone https://github.com/yoelrc88/Underwater-Color-Correction.git ; fi  
%cd Underwater-Color-Correction/

# !mkdir -p flowers/
# %cd flowers/
# !wget -nc https://s3.amazonaws.com/content.udacity-data.com/nd089/flower_data.tar.gz
# !if ! test -d test/; then tar -xzf flower_data.tar.gz; fi  



/content
/content/Underwater-Color-Correction




```
# This is formatted as code
```

### Google Drive

If something we need to download something from a google drive link (models, datasets)

In [0]:
# !pip install -U -q PyDrive

# from pydrive.auth import GoogleAuth
# from pydrive.drive import GoogleDrive
# from google.colab import auth
# from oauth2client.client import GoogleCredentials
# auth.authenticate_user()
# gauth = GoogleAuth()
# gauth.credentials = GoogleCredentials.get_application_default()
# drive = GoogleDrive(gauth)

## Run 

In [7]:
!python ./eval_one.py

You must provide an info.pkl file and an image
